In [1]:
import sys
sys.path.insert(0, '/home/louchao/project/semantic_parsing_qcfg/')
from src.models.tgt_parser.struct.pcfg_rdp import PCFGRandomizedDP, XYZ, XYz
from src.models.tgt_parser.struct.pcfg import PCFG
import torch

In [2]:

torch.random.manual_seed(1)

B, N, T, NT = 1, 60, 80, 80
device = "cuda"
params = {
    "term": torch.randn(B, N, T, device=device)
    .log_softmax(-1)
    .requires_grad_(True),
    "root": torch.randn(B, NT, device=device).log_softmax(-1).requires_grad_(True),
    "rule": torch.randn(B, NT, (NT + T) ** 2, device=device)
    .log_softmax(-1)
    .view(B, NT, NT + T, NT + T)
    .requires_grad_(True),
}
lens = torch.tensor([N], dtype=torch.long, device=device)

# sanity check 
pcfg = PCFGRandomizedDP(3, 3, 0)
out = torch.zeros(B, device=device)
print(pcfg(params, lens))
for _ in range(100):
    out += pcfg(params, lens)
print(out / 100)

pcfg = PCFG()
print(pcfg(params, lens))
print(pcfg(params, lens, decode=True))


In [3]:
from pytorch_memlab import LineProfiler

In [4]:
pcfg = PCFGRandomizedDP(10, 10, 0)
with LineProfiler(pcfg.__call__, XYZ, XYz) as prof:
    pcfg(params, lens, decode=True)
prof.display()

/home/louchao/anaconda3/envs/neural-qcfg/lib/python3.9/site-packages/pytorch_memlab/line_profiler/line_records.py:60: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  records = (_accumulate_line_records(raw_line_records)
/home/louchao/anaconda3/envs/neural-qcfg/lib/python3.9/site-packages/pytorch_memlab/line_profiler/line_records.py:189: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged = merged.drop('code', 1, level=0)
/home/louchao/anaconda3/envs/neural-qcfg/lib/python3.9/site-packages/pytorch_memlab/line_profiler/line_records.py:189: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  merged = merged.drop('code', 1, level=0)
/home/louchao/anaconda3/envs/neural-qcfg/lib/python3.9/site-packages/pytorch_memlab/line_profiler/line_rec

active_bytes,reserved_bytes,line,code
all,all,,
peak,peak,,
7.83M,22.00M,20,"def __call__(self, params, lens, decode=False, marginal=False):"
,,21,# terms: bsz x seqlen x pt
,,22,# rules: bsz x nt x (nt+pt) x (nt+pt)
,,23,# roots: bsz x nt
,,24,
7.83M,22.00M,25,"topk, sample_size, smooth = self.topk, self.sample_size, self.smooth"
7.83M,22.00M,26,"batch, N, T = params[""term""].shape"
7.83M,22.00M,27,N += 1


In [10]:
pcfg(params, lens, decode=True)
torch.cuda.max_memory_allocated(device=None) / 1024 / 1024

136.662109375